In [ ]:
# install epitran to extract phonemes
!pip install epitran

In [ ]:
!pip install eng-to-ipa 

     |████████████████████████████████| 2.8 MB 5.0 MB/s 
  Created wheel for eng-to-ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822641 sha256=12a2e2826e03b09486ae8365ee6ac675e14cd178c846360d787c37d464375415
  Stored in directory: /root/.cache/pip/wheels/96/c0/dd/aeddfbebc2c3301c3dd09670d9954b0574ac4cd982664c1110
Successfully built eng-to-ipa


In [ ]:
import eng_to_ipa as ipa
import epitran

In [ ]:
import math
import pandas as pd
import string
import re
import numpy as np
from collections import Counter

Preprocessing for test file

In [ ]:
test_words = {}
with open('test_transcribed.txt','r') as file:
  test_file = file.readlines()

  for i in test_file:
    temp = i.split('\t')
    test_words[temp[0].strip()] = temp[1].strip()

In [ ]:
print(test_words.values())

dict_values(['ʁɑɪ̯n\n', 'giɐ\n', 'liʊ̯\n', 'aɐ̯\n', 'ʁɛgdi\n', 'gɒd\n', 'fɒ\n', 'mɑŋə\n', 'teŋ\n', 'mɛn\n', 'i\n', 'smɔ:\n', 'mɛŋdɐ\n', 'de\n', 'kan\n', 'vi\n', 'vesd\n', 'alə\n', 'bli:ʊ̯ə\n', 'e:niə\n', 'ɒm\n', 'ɛfdɐ\n', 'en\n', 'sɒmɐ\n', 'mɛð\n', 'kloɑgvan\n', 'kɛlʁɐnə\n', 'ɒʊ̯\n', 'ɒʊ̯ɐsvœməðə\n', 'mɑ:gɐ\n', 'sɒ\n', 'lɑŋd\n', 'ɒɪ̯əð\n', 'ʁagɐ\n', 'sdɒ:\n', 'li:ə\n', 'nu\n', 'ɒʊ̯ɐ\n', 'gigantisg\n', 'uðfɒ:dʁɛŋ\n', 'ad\n', 'telpasə\n', 'ɒs\n', 'fʁamtiðəns\n', 'kli:ma\n', 'hɒnte:ɐ\n', 'di\n', 'sdo:ɐ\n', 'ʁɑɪ̯nmɛŋdɐ\n', 'jɑɪ̯\n', 'tʁoɐ̯\n', 'hɛld\n', 'ɑʊ̯gœ:ɐnə\n', 'gɒ:\n', 'tel\n', 'ɑ:bɑɪ̯dəð\n', 'ɔ:bənheð\n', 'ɒʊ̯ɐfɒ\n', 'ny:ə\n', 'idɐ\n', 'pɑ:dnɐsgæ:bɐ\n', 'enən\n', 'komu:nə\n', 'endəð\n', 'fɒsy:neŋssɛlsgæ:b\n', 'bɒ:ʊ̯ɐ\n', 'lø:sə\n', 'hɛɐ̯\n', 'pʁoble:m\n', 'ale:nə\n', 'hɑ:\n', 'dɛɐ̯fɒ\n', 'vɛ:ɐð\n', 'mɑɪ̯əð\n', 'vegdid\n', 'mɑɪ̯\n', 'kunə\n', 'kɒmə\n', 'dæ:\n', 'sdɑ:də\n', 'dialo:ən\n', 'jaɐ̯\n', 'kɒmɐ\n', 'egə\n', 'dɛn\n', 'po:sə\n', 'pɛŋə\n', 'sɒm\n', 'no:ən\n', 'a\n', 'mɔsge:\n'

In [ ]:
print(len(test_words))

1858


In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head()

,language,sentence_id,word_id,word,ipa
0,da,CopCo-1,1,Regn,Regn
1,da,CopCo-1,2,giver,giver
2,da,CopCo-1,3,liv.,liv.
3,da,CopCo-2,4,Regn,Regn
4,da,CopCo-2,5,er,er


In [ ]:
test_df_words = test_df['word'].tolist()
test_ipa = []
for i in test_df_words:
  word = test_words.get(i)
  if word != None:
    test_ipa.append(word)
  else:
    test_ipa.append(i)

In [ ]:
test_df['ipa'] = test_ipa
test_df.to_csv('test.csv',index=False)

In [ ]:
test_df = pd.read_csv("test.csv")
test_df.head(10)

,language,sentence_id,word_id,word,ipa
0,da,CopCo-1,1,Regn,Regn
1,da,CopCo-1,2,giver,giɐ
2,da,CopCo-1,3,liv.,liv.
3,da,CopCo-2,4,Regn,Regn
4,da,CopCo-2,5,er,aɐ̯
5,da,CopCo-2,6,rigtig,ʁɛgdi
6,da,CopCo-2,7,godt,gɒd
7,da,CopCo-2,8,for,fɒ
8,da,CopCo-2,9,mange,mɑŋə
9,da,CopCo-2,10,ting,teŋ


In [ ]:
# PUT FILE CMCL FILE HERE TO EXTRACT FEATURES
input_file = "test.csv"

## Extract IPA Phonemes

Requirements: Install epitran package for str -> IPA form.

In [ ]:
train_df = pd.read_csv(input_file)
train_df.head()

,language,sentence_id,word_id,word
0,en,ZuCo1-3,0.0,Ford
1,en,ZuCo1-3,1.0,was
2,en,ZuCo1-3,2.0,forced
3,en,ZuCo1-3,3.0,out
4,en,ZuCo1-3,4.0,of


In [ ]:
ipa_list = []
lang_list = train_df['language'].tolist()
word_list = train_df['word'].tolist()

for i,j in zip(lang_list, word_list):
  if i == 'en':
    print(i,j)
    ipa_conv = ipa.convert(j)
    ipa_list.append(ipa_conv)
  elif i == 'zh':
    print(i,j)
    epi = epitran.Epitran('cmn-Hans',cedict_file='cedict_1_0_ts_utf-8_mdbg.txt')
    ipa_conv = epi.transliterate(j)
    ipa_list.append(ipa_conv)
  elif i == 'ru':
    print(i,j)
    epi = epitran.Epitran('rus-Cyrl')
    ipa_conv = epi.transliterate(j)
    ipa_list.append(ipa_conv)
  elif i == 'hi':
    print(i,j)
    epi = epitran.Epitran('hin-Deva')
    ipa_conv = epi.transliterate(j)
    ipa_list.append(ipa_conv)
  elif i == 'de':
    print(i,j)
    epi = epitran.Epitran('deu-Latn')
    ipa_conv = epi.transliterate(j)
    ipa_list.append(ipa_conv)
  elif i == 'nl':
    print(i,j)
    epi = epitran.Epitran('nld-Latn')
    ipa_conv = epi.transliterate(j)
    ipa_list.append(ipa_conv)

Streaming output truncated to the last 5000 lines.
en In
en 1964
en she
en went
en to
en Reprise
en again,
en shifting
en the
en next
en year
en to
en Dot
en Records.
en In
en 1966
en she
en went
en to
en United
en Artists
en Records.
en Michael
en McManus
en (born
en February
en 5,
en 1959)
en is
en a
en syndicated
en columnist
en who
en wrote
en Ethics
en &
en Religion.
en On
en January
en 28,
en 2005
en it
en was
en revealed
en that
en he
en accepted
en money
en from
en the
en George
en W.
en Bush
en administration
en to
en promote
en their
en marriage
en initiative
en program,
en which
en he
en did
en not
en disclose
en to
en his
en readers.
en Through
en his
en son
en Timothy
en Bush,
en Jr.,
en who
en was
en also
en a
en blacksmith,
en descended
en two
en American
en Presidents
en -George
en H.
en W.
en Bush
en and
en George
en W.
en Bush.
en Anne
en Hutchinson
en (July
en 17,
en 1591
en -
en August
en 20,
en 1643)
en was
en the
en unauthorized
en Puritan
en preacher
en of
en a
e

In [ ]:
train_df['ipa'] = ipa_list
train_df.to_csv(input_file,index=False)

## Extract Length and Entropy

Requirements: IPA forms of words.

In [ ]:
train_df = pd.read_csv(input_file)
train_df.head()

,language,sentence_id,word_id,word,ipa
0,da,CopCo-1,1,Regn,Regn
1,da,CopCo-1,2,giver,giɐ
2,da,CopCo-1,3,liv.,liv.
3,da,CopCo-2,4,Regn,Regn
4,da,CopCo-2,5,er,aɐ̯


In [ ]:
# functions for entropy
def entropy(s):
  p, lns = Counter(s), float(len(s))
  return -sum(count/lns * math.log(count/lns, 2) for count in p.values())

In [ ]:
len_list = []
ent_list = []
let_len_list = []
ipa_list = train_df['ipa'].tolist()

for i in ipa_list:
  i = str(i)
  ipa_len = len(i.replace(r'[^\w]',''))
  ipa_ent = entropy(i.replace(r'[^\w]',''))
  #print(ipa_len, ipa_ent)
  len_list.append(ipa_len)
  ent_list.append(ipa_ent)

for i in train_df['word'].tolist():
  let_len = len(i.replace(r'[^\w]',''))
  let_len_list.append(let_len)

In [ ]:
train_df['ipa_len'] = len_list
train_df['ipa_ent'] = ent_list
train_df['word_len'] = let_len_list
train_df.to_csv(input_file,index=False)

## Extract IPA Vowel Count and Density

Requirements: IPA forms of words.

In [ ]:
train_df = pd.read_csv(input_file)
ipa_list = train_df['ipa'].tolist()

In [ ]:
ipa_vowels = ['a','i','ɪ','e','ɛ','æ','ʌ','ə','u','ʊ','o','ɔ','	iː','aː','uː','eː','oː','ɛː','œ','ɐ','yː','øː','ʏ','ɔː','ɪ̃','ĩː','ũː','ã̈ː','ɔ̃ː','äː','ɛ̃ː','ẽː','õː','ə̃','ɤ','ə˞','a˞','u˞','ɑ̃˞','a̟˞','ɤ˞','ə̃˞','aɪ','aʊ','ɔɪ','oʊ','eɪ̯','iɪ','ɚː']
ipa_vowel_density_list = []
ipa_vowel_count_list = []

for i in ipa_list:
  i = str(i)
  ipa_len = len(i.replace(r'[^\w]',''))
  ipa_vowel_count = 0
  for vowel in ipa_vowels:
    if vowel in i:
      ipa_vowel_count += 1
  if ipa_vowel_count > 0:
    vowel_density = ipa_vowel_count / ipa_len
  ipa_vowel_count_list.append(ipa_vowel_count)
  ipa_vowel_density_list.append(vowel_density)

In [ ]:
train_df['ipa_vowel_density'] = ipa_vowel_density_list
train_df['ipa_vowel_count'] = ipa_vowel_count_list
train_df.to_csv(input_file,index=False)

## Extract Imageability and Concreteness

Requirement: Dictionary files of word embedding-based imageability and concreteness scores from Ljubesic et al (2018) for 77 languages. Link: http://hdl.handle.net/11356/1187

In [ ]:
train_df = pd.read_csv(input_file)
train_df.head()

,language,sentence_id,word_id,word,ipa,ipa_len,ipa_ent,word_len,ipa_vowel_density,ipa_vowel_count
0,da,CopCo-1,1,Regn,Regn,4,2.000000,4,0.250000,1
1,da,CopCo-1,2,giver,giɐ,3,1.584963,5,0.666667,2
2,da,CopCo-1,3,liv.,liv.,4,2.000000,4,0.250000,1
3,da,CopCo-2,4,Regn,Regn,4,2.000000,4,0.250000,1
4,da,CopCo-2,5,er,aɐ̯,3,1.584963,2,0.666667,2


In [ ]:
lang_list = ['da']
#lang_list = ['en','zh','ru','hi','de','nl']

In [ ]:
from statistics import mean

concreteness_list_master = []
imageability_list_master = []

for lang in lang_list:
  print(lang)
  train_lang_select = train_df.loc[train_df['language'] == lang]
  word_list = []

  # read imageability and concreteness file
  imageability_dict = {}
  concreteness_dict = {}
  file_name = 'megahr.'+lang+'.txt'
  with open(file_name,'r') as f:
    content = f.readlines()
    for entry in content:
      entry_list = entry.split('\t')
      word_list.append(entry_list[0])
      concreteness_dict[entry_list[0]] = float(entry_list[1])
      imageability_dict[entry_list[0]] = float(entry_list[2])

  # get concreteness and imageability values from dictionaries
  for word in train_lang_select['word'].tolist():
    word = word.replace(r'[^\w]','').lower()
    # if word in training data is existing in the external wordlist of imageability and concreteness
    if word in word_list:
      concreteness_list_master.append(concreteness_dict[word])
      imageability_list_master.append(imageability_dict[word])
    else:
      concreteness_list_master.append(0)
      imageability_list_master.append(0)

  concreteness_mean = mean(concreteness_dict[k] for k in concreteness_dict)
  imageability_mean = mean(imageability_dict[k] for k in imageability_dict)
  
  # if word is not present, substitute with mean
  for i in range(len(concreteness_list_master)):
    if concreteness_list_master[i] == 0:
      concreteness_list_master[i] = concreteness_mean

  for i in range(len(imageability_list_master)):
    if imageability_list_master[i] == 0:
      imageability_list_master[i] = imageability_mean


da


In [ ]:
print(len(concreteness_list_master))
print(len(imageability_list_master))

6767
6767


In [ ]:
train_df['concreteness'] = concreteness_list_master
train_df['imageability'] = imageability_list_master
train_df.to_csv(input_file,index=False)

## Extract Bigram and Trigram Density

Requirements: Character bigram and trigram extracted from CMCL data in sentence form.

In [ ]:
from collections import Counter

train_df = pd.read_csv(input_file)
train_df.head()

,language,sentence_id,word_id,word,ipa,ipa_len,ipa_ent,word_len,ipa_vowel_density,ipa_vowel_count,concreteness,imageability
0,da,CopCo-1,1,Regn,Regn,4,2.000000,4,0.250000,1,4.067915,4.415752
1,da,CopCo-1,2,giver,giɐ,3,1.584963,5,0.666667,2,3.387675,3.745939
2,da,CopCo-1,3,liv.,liv.,4,2.000000,4,0.250000,1,3.512695,3.828328
3,da,CopCo-2,4,Regn,Regn,4,2.000000,4,0.250000,1,4.067915,4.415752
4,da,CopCo-2,5,er,aɐ̯,3,1.584963,2,0.666667,2,3.298299,3.601879


Bigram

In [ ]:
bigram_df = pd.read_csv('bigram_dict.csv')
bigram_word = bigram_df['bigram'].tolist()
bigram_freq = bigram_df['freq'].tolist()

bigram_dict = {}
for i,j in zip(bigram_word,bigram_freq):
  bigram_dict[i] = j

In [ ]:
bigram_count = []
bigram_density_sum = []
bigram_density_norm = []
ipa_list = train_df['ipa'].tolist()

for i in ipa_list:
  i = str(i)
  word_len = len(i)
  bigram_sum = 0
  bigram_div = 0
  if i != "":
    ipa_chars = Counter(i[idx : idx + 2] for idx in range(len(i) - 1))
    bigram_count.append(len(ipa_chars.keys()))
    for gram in ipa_chars.keys():
      for key,value in bigram_dict.items():
        if gram == key:
          bigram_sum +=value
  
  # append bigram sum
  bigram_density_sum.append(bigram_sum)
  
  # append bigram sum normalized with # of bigrams
  if (word_len != 0) and (bigram_sum != 0):
    bigram_div = bigram_sum/word_len
  bigram_density_norm.append(bigram_div)


In [ ]:
print(len(ipa_list))
print(len(bigram_count))
print(len(bigram_density_sum))
print(len(bigram_density_norm))

6767
6767
6767
6767


In [ ]:
train_df['bigram_count'] = bigram_count
train_df['bigram_sum'] = bigram_density_sum
train_df['bigram_norm'] = bigram_density_norm
train_df.to_csv(input_file,index=False)

Trigram

In [ ]:
trigram_df = pd.read_csv('trigram_dict.csv')
trigram_word = trigram_df['trigram'].tolist()
trigram_freq = trigram_df['freq'].tolist()

trigram_dict = {}
for i,j in zip(trigram_word,trigram_freq):
  trigram_dict[i] = j

In [ ]:
trigram_count = []
trigram_density_sum = []
trigram_density_norm = []
ipa_list = train_df['ipa'].tolist()

for i in ipa_list:
  i = str(i)
  word_len = len(i)
  trigram_sum = 0
  trigram_div = 0
  if i != "":
    ipa_chars = Counter(i[idx : idx + 3] for idx in range(len(i) - 1))
    trigram_count.append(len(ipa_chars.keys()))
    for gram in ipa_chars.keys():
      for key,value in trigram_dict.items():
        if gram == key:
          trigram_sum +=value
  
  # append trigram sum
  trigram_density_sum.append(trigram_sum)
  
  # append trigram sum normalized with # of trgrams
  if (word_len != 0) and (trigram_sum != 0):
    trigram_div = trigram_sum/word_len
  trigram_density_norm.append(trigram_div)


In [ ]:
print(len(ipa_list))
print(len(trigram_count))
print(len(trigram_density_sum))
print(len(trigram_density_norm))

6767
6767
6767
6767


In [ ]:
train_df['trigram_count'] = trigram_count
train_df['trigram_sum'] = trigram_density_sum
train_df['trigram_norm'] = trigram_density_norm
train_df.to_csv(input_file,index=False)

In [ ]:
train_df.head()

,language,sentence_id,word_id,word,ipa,ipa_len,ipa_ent,word_len,ipa_vowel_density,ipa_vowel_count,concreteness,imageability,bigram_count,bigram_sum,bigram_norm,trigram_count,trigram_sum,trigram_norm
0,da,CopCo-1,1,Regn,Regn,4,2.000000,4,0.250000,1,4.067915,4.415752,3,6,1.50,3,0,0.0
1,da,CopCo-1,2,giver,giɐ,3,1.584963,5,0.666667,2,3.387675,3.745939,2,6,2.00,2,0,0.0
2,da,CopCo-1,3,liv.,liv.,4,2.000000,4,0.250000,1,3.512695,3.828328,3,519,129.75,3,8,2.0
3,da,CopCo-2,4,Regn,Regn,4,2.000000,4,0.250000,1,4.067915,4.415752,3,6,1.50,3,0,0.0
4,da,CopCo-2,5,er,aɐ̯,3,1.584963,2,0.666667,2,3.298299,3.601879,2,0,0.00,2,0,0.0


## Extract Unigram Probability and Phonotactic Complexity

Requirements: Sentence form of CMCL data. Train an LM from the train data and use -log(probability of word|context).

In [ ]:
train_df = pd.read_csv(input_file)
ipa_list = train_df['ipa'].tolist()
train_df.head()

,language,sentence_id,word_id,word,ipa,ipa_len,ipa_ent,word_len,ipa_vowel_density,ipa_vowel_count,concreteness,imageability,bigram_count,bigram_sum,bigram_norm,trigram_count,trigram_sum,trigram_norm
0,da,CopCo-1,1,Regn,Regn,4,2.000000,4,0.250000,1,4.067915,4.415752,3,6,1.50,3,0,0.0
1,da,CopCo-1,2,giver,giɐ,3,1.584963,5,0.666667,2,3.387675,3.745939,2,6,2.00,2,0,0.0
2,da,CopCo-1,3,liv.,liv.,4,2.000000,4,0.250000,1,3.512695,3.828328,3,519,129.75,3,8,2.0
3,da,CopCo-2,4,Regn,Regn,4,2.000000,4,0.250000,1,4.067915,4.415752,3,6,1.50,3,0,0.0
4,da,CopCo-2,5,er,aɐ̯,3,1.584963,2,0.666667,2,3.298299,3.601879,2,0,0.00,2,0,0.0


In [ ]:
with open('cmcl_ipa_sentences.txt','r') as file:
  cmcl_ipa_sents = file.readlines()
  cmcl_ipa_sents = [x.strip() for x in cmcl_ipa_sents]
  print(len(cmcl_ipa_sents))

1695


In [ ]:
cmcl_sents_df = pd.DataFrame(list(zip(cmcl_ipa_sents)),columns =['sentence'])

cmcl_sents_df['sentence_clean'] = cmcl_sents_df['sentence'].str.replace('[{}]'.format(string.punctuation), '')
cmcl_sents_df['sentence_clean'] = cmcl_sents_df['sentence'].str.lower()

cmcl_sents_df['sentence_clean'] = '<s ' + cmcl_sents_df['sentence_clean']
cmcl_sents_df['sentence_clean'] = cmcl_sents_df['sentence_clean'] + ' /s>'

cmcl_sents_df.head()

,sentence,sentence_clean
0,"wɪθ hɪz ˈɪntəˌrɛst ɪn reɪs kɑz, hi fɔrmd ə ˈsɛ...","<s wɪθ hɪz ˈɪntəˌrɛst ɪn reɪs kɑz, hi fɔrmd ə ..."
1,"ˈdʊrɪŋ ðɪs ˈpɪriəd, hi ˈpərsənəli droʊv hɪz qu...","<s ˈdʊrɪŋ ðɪs ˈpɪriəd, hi ˈpərsənəli droʊv hɪz..."
2,ðiz ɪkˈspɛrəmənts ˈkəlməˌneɪtɪd ɪn 1896 wɪθ ðə...,<s ðiz ɪkˈspɛrəmənts ˈkəlməˌneɪtɪd ɪn 1896 wɪθ...
3,fɔrd wɑz bɔrn ɔn ə ˈprɑspərəs fɑrm ɪn springwe...,<s fɔrd wɑz bɔrn ɔn ə ˈprɑspərəs fɑrm ɪn sprin...
4,ˈhɛnri fɔrd ˈædvəˌkeɪtɪd long-time* əˈsoʊʃiˌeɪ...,<s ˈhɛnri fɔrd ˈædvəˌkeɪtɪd long-time* əˈsoʊʃi...


In [ ]:
cmcl_whole_text = cmcl_sents_df['sentence_clean']
cmcl_text = " ".join(map(str, cmcl_whole_text))
cmcl_text

'<s wɪθ hɪz ˈɪntəˌrɛst ɪn reɪs kɑz, hi fɔrmd ə ˈsɛkənd ˈkəmpəˌni, ðə ˈhɛnri fɔrd ˈkəmpəˌni. /s> <s ˈdʊrɪŋ ðɪs ˈpɪriəd, hi ˈpərsənəli droʊv hɪz quadricycle* tɪ ˈvɪktəri ɪn ə reɪs əˈgɛnst ˌælɪgˈzændər ˈwɪntən, ə ˈwɛlˈnoʊn ˈdraɪvər ənd ðə ˈhɛvi ˈfeɪvərɪt ɔn ɑkˈtoʊbər 10 1901 /s> <s ðiz ɪkˈspɛrəmənts ˈkəlməˌneɪtɪd ɪn 1896 wɪθ ðə kəmˈpliʃən əv hɪz oʊn self-propelled* ˈviɪkəl neɪmd ðə quadricycle*, wɪʧ hi test-drove* ɔn ʤun 4 əv ðət jɪr. /s> <s fɔrd wɑz bɔrn ɔn ə ˈprɑspərəs fɑrm ɪn springwells* ˈtaʊnʃɪp (naʊ ɪn ðə ˈsɪti əv ˈdɪrˌbɔrn, ˈmɪʃɪgən) oʊnd baɪ hɪz ˈpɛrənts, ˈwɪljəm fɔrd 1826-1905* ənd ˈmɛri litogot* (c1839-1876* ˈɪməgrənts frəm ˈkaʊnti kɔrk, ˈaɪərlənd. /s> <s ˈhɛnri fɔrd ˈædvəˌkeɪtɪd long-time* əˈsoʊʃiˌeɪt ˈhɛri ˈbɛnɪt tɪ teɪk ðə spɑt. /s> <s edsel\'s* ˈwɪdoʊ ˈɛlənər, hu hæd ˌɪnˈhɛrətɪd edsel\'s* ˈvoʊtɪŋ stɑk, ˈwɔntɪd hər sən ˈhɛnri fɔrd ii* tɪ teɪk ˈoʊvər ðə pəˈzɪʃən. /s> <s ɪn 1879 hi lɛft hoʊm fər ðə ˈnɪrˈbaɪ ˈsɪti əv ˈdiˌtrɔɪt tɪ wərk ɛz ən əˈprɛntɪs məˈʃinəst, fərst wɪθ ʤeɪmz ɛ

In [ ]:
cmcl_text_list = cmcl_text.split(' ')
cmcl_text_list = [re.sub(r'[, .\'\"]','',x) for x in cmcl_text_list]
cmcl_text_list

['<s',
 'wɪθ',
 'hɪz',
 'ˈɪntəˌrɛst',
 'ɪn',
 'reɪs',
 'kɑz',
 'hi',
 'fɔrmd',
 'ə',
 'ˈsɛkənd',
 'ˈkəmpəˌni',
 'ðə',
 'ˈhɛnri',
 'fɔrd',
 'ˈkəmpəˌni',
 '/s>',
 '<s',
 'ˈdʊrɪŋ',
 'ðɪs',
 'ˈpɪriəd',
 'hi',
 'ˈpərsənəli',
 'droʊv',
 'hɪz',
 'quadricycle*',
 'tɪ',
 'ˈvɪktəri',
 'ɪn',
 'ə',
 'reɪs',
 'əˈgɛnst',
 'ˌælɪgˈzændər',
 'ˈwɪntən',
 'ə',
 'ˈwɛlˈnoʊn',
 'ˈdraɪvər',
 'ənd',
 'ðə',
 'ˈhɛvi',
 'ˈfeɪvərɪt',
 'ɔn',
 'ɑkˈtoʊbər',
 '10',
 '1901',
 '/s>',
 '<s',
 'ðiz',
 'ɪkˈspɛrəmənts',
 'ˈkəlməˌneɪtɪd',
 'ɪn',
 '1896',
 'wɪθ',
 'ðə',
 'kəmˈpliʃən',
 'əv',
 'hɪz',
 'oʊn',
 'self-propelled*',
 'ˈviɪkəl',
 'neɪmd',
 'ðə',
 'quadricycle*',
 'wɪʧ',
 'hi',
 'test-drove*',
 'ɔn',
 'ʤun',
 '4',
 'əv',
 'ðət',
 'jɪr',
 '/s>',
 '<s',
 'fɔrd',
 'wɑz',
 'bɔrn',
 'ɔn',
 'ə',
 'ˈprɑspərəs',
 'fɑrm',
 'ɪn',
 'springwells*',
 'ˈtaʊnʃɪp',
 '(naʊ',
 'ɪn',
 'ðə',
 'ˈsɪti',
 'əv',
 'ˈdɪrˌbɔrn',
 'ˈmɪʃɪgən)',
 'oʊnd',
 'baɪ',
 'hɪz',
 'ˈpɛrənts',
 'ˈwɪljəm',
 'fɔrd',
 '1826-1905*',
 'ənd',
 'ˈmɛri',
 'litogot

In [ ]:
phonetic_comp_list = []
for i in ipa_list:
  indices = [a for a, x in enumerate(cmcl_text_list) if x == i]
  #print(indices)
  phone_complexity_sum = 0
  phone_complexity_norm = 0
  for j in indices:
    if cmcl_text_list[j-1] != '<s':
      try:
        word_prob = len(re.findall(re.sub('[\[\(\)]', '', cmcl_text_list[j-1]) +' '+ re.sub('[\[\(\)]','',cmcl_text_list[j]), cmcl_text)) / len(re.findall(re.sub('[\[\(\)]','',cmcl_text_list[j]), cmcl_text))
      except ZeroDivisionError:
        word_prob = 0
      if word_prob != 0:
        phone_complexity = -(math.log(word_prob,2))
      else:
        phone_complexity = 0
      phone_complexity_sum += phone_complexity
    else:
      word_prob = len(re.findall(re.sub('[\[\(\)]','',cmcl_text_list[j]), cmcl_text)) / len(cmcl_text_list)
      phone_complexity = -(math.log(word_prob,2))
  if len(indices) != 0:
    phone_complexity_norm = phone_complexity_sum/len(indices)
  print(phone_complexity_norm)
  phonetic_comp_list.append(phone_complexity_norm)

Streaming output truncated to the last 5000 lines.
0.0
0
0
0
3.11440934524794
11.432532719398194
0
0
3.11440934524794
3.9118439248435704
0
11.432532719398194
0
11.432532719398194
0
0
0
0
0
1.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4.0
0
0
0
0
11.432532719398194
0
0
3.11440934524794
0
0
0
0
3.11440934524794
0
0
0
0
0
0
0
0
0
0
11.432532719398194
0
0
0
0
3.11440934524794
0
0
0
0
0
0
0
0
0.0
0
11.432532719398194
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
11.432532719398194
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
7.624121311829187
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
11.432532719398194
0
0
0
0
0
0
0
0
3.11440934524794
0
0
0
0
0
3.11440934524794
0
0
0
0
0
4.0
3.11440934524794
0
0
0
0
0
0
0
0
0
0
0
11.432532719398194
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
11.432532719398194
0
0
0
0
0
0
0
0
0
0
11.432532719398194
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3.11440934524794
11.4325327

In [ ]:
print(len(phonetic_comp_list))

6767


In [ ]:
train_df['phonetic_comp'] = phonetic_comp_list
train_df.to_csv(input_file,index=False)

In [ ]:
train_df.head()

,language,sentence_id,word_id,word,ipa,ipa_len,ipa_ent,word_len,ipa_vowel_density,ipa_vowel_count,concreteness,imageability,bigram_count,bigram_sum,bigram_norm,trigram_count,trigram_sum,trigram_norm,phonetic_comp
0,da,CopCo-1,1,Regn,Regn,4,2.000000,4,0.250000,1,4.067915,4.415752,3,6,1.50,3,0,0.0,0.0
1,da,CopCo-1,2,giver,giɐ,3,1.584963,5,0.666667,2,3.387675,3.745939,2,6,2.00,2,0,0.0,0.0
2,da,CopCo-1,3,liv.,liv.,4,2.000000,4,0.250000,1,3.512695,3.828328,3,519,129.75,3,8,2.0,0.0
3,da,CopCo-2,4,Regn,Regn,4,2.000000,4,0.250000,1,4.067915,4.415752,3,6,1.50,3,0,0.0,0.0
4,da,CopCo-2,5,er,aɐ̯,3,1.584963,2,0.666667,2,3.298299,3.601879,2,0,0.00,2,0,0.0,0.0
